In [1]:
from sage.symbolic.expression import Expression
from delierium.MatrixOrder import Context, Mlex, Mgrlex, Mgrevlex
from IPython.display import Math
from delierium.exception import DelieriumNotALinearPDE
from dataclasses import dataclass
from delierium.helpers import latexer
from sage.symbolic.expression_conversions import ExpressionTreeWalker
from IPython.core.debugger import set_trace
import doctest
from typing import Union, Any, Optional, TYPE_CHECKING, cast
from functools import partial, wraps, cache, total_ordering
from operator import mul
from IPython.display import Math
from collections import Counter
from pprint import pprint

In [2]:
x, y= var("x y")
xi  = function("xi", latex_name=r"\xi")(x,y)
eta = function("eta", latex_name = r"\eta")(x, y)
ctx = Context([xi, eta], [x, y], Mgrevlex)
xi_x = diff(xi, x)
xi_y = diff(xi, y)
xi_xx = diff(xi, x,x)
xi_yy = diff(xi, y,y)
xi_xy = diff(xi, x,y)
eta_x = diff(eta, x)
eta_y = diff(eta, y)
eta_xx = diff(eta, x,x)
eta_yy = diff(eta, y,y)
eta_xy = diff(eta, x,y)

A = function("A")(x,y)
B = function("B")(x,y)
C = function("C")(x,y)
D = function("D")(x,y)
A_x = diff(A, x)
A_y = diff(A, y)
B_x = diff(B, x)
B_y = diff(B, y)
C_x = diff(C, x)
C_y = diff(C, y)
D_x = diff(D, x)
D_y = diff(D, y)

In [3]:
e1 = xi_yy - 2*A*eta_y - B*xi_y + A*xi_x - A_y*eta-A_x*xi
e2 = eta_xx - D*eta_y +C*eta_x +2*D*xi_x +D_y*eta+D_x*xi
e3 = eta_xy - xi_xx/2 +B*eta_x+3*D*xi_y/2+C*xi_x/2 + C_y*eta/2 + C_x*xi/2
e4 = eta_yy - 2*xi_xy +B*eta_y + 3*A*eta_x + 2*C*xi_y + B_y*eta +B_x*xi

In [4]:
#def contextualize(func):\n",
#    @wraps(function)\n",
#    def context_wrapper(context, *args):\n",
#        return partial(func, context)\n",
#    return context_wrapper"

In [5]:
_Sage_Expression = sage.symbolic.expression.Expression
_Delierium_Context = Context
def gen_summands(context: _Delierium_Context, expression: _Sage_Expression) -> list[_Sage_Expression]:
    try :
        
        if expression.operator().__name__ == 'add_vararg':
            # XXX: use expression.iterator
            return expression.operands()
    except AttributeError:
        return [expression]
    return [expression]

In [6]:
@total_ordering
@dataclass
class DT:
    context: Context
    expression: _Sage_Expression
    coefficient: _Sage_Expression
    derivative: _Sage_Expression
    function: _Sage_Expression
    order: tuple[int]
    comparison_order: tuple[int]
    def __eq__(self, other):
        return self.comparison_order == other.comparison_order
    def __gt__(self, other):
        return self.context.gt(self.comparison_order,other.comparison_order)
    def __lt__(self, other):
        return self.comparison_order != other.comparison_order and \
            not self.context.gt(self.comparison_order,other.comparison_order)
    def __str__(self):
        if str(self.coefficient) in ("", "1"):
            return f"{self.derivative}"
        return f"{self.coefficient} * {self.derivative}"
    def __key(self):
        return self.coefficient, self.derivative
    def __hash__(self):
        return hash(self.__key())

In [7]:
def is_derivative(e: Any) -> bool:
    """
    Check if an expression is a pure derivative
    >>> x, y= var("x y")
    >>> xi  = function("xi")(x,y)
    >>> ctx = Context([xi], [x,y])
    >>> xi_x = diff(xi, x)
    >>> xi_y = diff(xi, y)
    >>> xi_xx = diff(xi, x,x)
    >>> xi_yy = diff(xi, y,y)
    >>> xi_xy = diff(xi, x,y)
    >>> A = function("A")(x,y)
    >>> A_x = diff(A, x)
    >>> is_derivative(A)
    False
    >>> is_derivative(1)
    False
    >>> is_derivative(A*xi_x)
    False
    >>> is_derivative(xi)
    False
    >>> is_derivative(xi_xy)
    True
    >>> is_derivative(2*xi_x)
    False
    """
    try:
        return e.operator().__class__.__name__ == "FDerivativeOperator"
    except AttributeError:
        return False

doctest.testmod()

/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing Euler_Phi from here is deprecated; please use "from sage.arith.misc import Euler_Phi" instead.
See https://github.com/sagemath/sage/issues/30322 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: this class is deprecated, do not use
See https://github.com/sagemath/sage/issues/31545 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing Moebius from here is deprecated; please use "from sage.arith.misc import Moebius" instead.
See https://github.com/sagemath/sage/issues/30322 for details.
  return hasattr(f, '__wrapped__')
/home/tapir/mambaforge/envs/sage/lib/python3.11/inspect.py:753: DeprecationWarning: 
Importing OpenInterval from here is deprecated; please use "from sage.manifolds.differentiable.examples.real_line import Ope

TestResults(failed=0, attempted=16)

In [8]:
def order_of_derivative(context: _Delierium_Context, e: Any):
    '''Returns the vector of the orders of a derivative respect to its variables
    >>> x, y, z= var("x y z")
    >>> f1  = function(\"f1\")(x,y, z)
    >>> ctx = Context([f1], [x,y, z])
    >>> order_of_derivative(ctx, diff(f1, x, x, x, x))
    [4, 0, 0]
    >>> order_of_derivative(ctx, f1)
    [0, 0, 0]
    >>> order_of_derivative(ctx, diff(f1, x, x, x, y, z, z))
    [3, 1, 2]
    >>> ctx = Context([f1], [z, x, y])
    >>> order_of_derivative(ctx, diff(f1, x, x, x, x))
    [0, 4, 0]
    >>> order_of_derivative(ctx, diff(f1, x, x, x, y, z, z))
    [2, 3, 1]
    '''
    res = [0] * len(e.variables())
    if not is_derivative(e):
        return res
    for idx, variable in enumerate(e.variables()):
        # XXX. here seems to be the crucial part: order can depend on:
        # - either the order given by sage by
        # -- sage order
        # -- order given by the order given by function definition
        # - the order given by context
        i = context._independent.index(variable)
        res[i] = e.operator().parameter_set().count(idx)
    return res

doctest.testmod()

TestResults(failed=0, attempted=25)

In [9]:
def _compute_comparison_vector(context, func):
    """
    >>> x, y, z= var(\"x y z\")
    >>> f1  = function(\"f1\")(x, y, z)
    >>> f2  = function(\"f2\")(x, y, z)
    >>> f3  = function(\"f3\")(x, y, z)
    >>> ctx = Context([f1, f2, f3], [x, y, z])
    >>> _compute_comparison_vector(ctx, f1)
    [1, 0, 0]
    >>> _compute_comparison_vector(ctx, f3)
    [0, 0, 1]
    """
    iv = [0] * len(context._dependent)
    if func in context._dependent:
        iv[context._dependent.index(func)] = 1
    elif func.operator() in context._dependent:
        iv[context._dependent.index(func.operator())] = 1
    else:
        pass
    return iv

doctest.testmod()  

TestResults(failed=0, attempted=32)

In [10]:
def compute_order(context, expression):
    """computes the monomial tuple from the derivative part"""
    if is_derivative(expression):
        return order_of_derivative(expression, context,                    
                                   len(context._independent))
    # XXX: Check can that be within a system of linear PDEs ? I think not, as the function must then be identically to zero
    return [0] * len(context._independent)

In [11]:
def _gen_dt(context: _Delierium_Context, expression: _Sage_Expression) -> tuple[tuple[_Sage_Expression], tuple[_Sage_Expression]]:
    coeff = []
    derivs = []
    match expression.operator().__class__.__name__:
        case "FDerivativeOperator":
            if expression.operator().function() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "Expression":
            print("Symbolic Expression")
        case "NewSymbolicFunction":
            if expression.operator() in context._dependent:
                derivs.append(expression)
            else:
                coeff.append(expression)
        case "function":
            if expression.operator().__name__ == "mul_vararg":
                for operand in expression.operands():
                    if not operand.operator():
                        coeff.append(operand)
                    else:
                        c, d = _gen_dt(context, operand)
                        coeff.extend(c)
                        derivs.extend(d)
        case _:
            coeff.append(expression)
    return coeff, derivs

def gen_dt(context: _Delierium_Context, expression: _Sage_Expression)-> bool:
    expression = expression.expand()
    c, d = _gen_dt(context, expression)
    if not d:
        raise DelieriumNotALinearPDE(expression)
    d = reduce(mul, d, 1)
    order = order_of_derivative(context, d)
    # XXX here we have our main problem:
    # - if we change the order of the variables with context
    # - then the order of the context and the order of the variables
    # - as they are used in the functions defined by sagemath differ
    # - so all stuff depending on ordering the variables(and which is
    # - crucial for all our algorithms) gets screwed up. We have to store
    # --- the order as given by context
    # --- the order as given by the canonical sagemath
    # --- a variable which selects which order has to be used in each step
    return DT(context,
              expression,
              reduce(mul, c, 1), d,
              d.operator().function() if is_derivative(d) else d.function(),              
              order,
              order + 
              _compute_comparison_vector(context, d.operator().function() if is_derivative(d) else d.function())
             )

In [12]:
def latexer_dt(context, dt):
    def _latex_derivative(context, deriv):
        if is_derivative(deriv):
            func = deriv.function().operator().function()._latex_()
            ps = deriv.operator().parameter_set()
            variables = deriv.operands()
            sub = ",".join(map(lambda _ : variables[_]._latex_(), ps))
            return "%s_{%s}" % (func, sub)
        else:
            return deriv.function().operator()._latex_()
    def _latex_coeff(context, coeff):
        if str(coeff) == '1':
            return ""
        if str(coeff) == '-1':
            return "-"
        # need to be more fine granular
        c = coeff.expand().simplify_full()._latex_() 
        if hasattr(coeff, "operator") and \
            coeff.operator() != None and \
            coeff.operator().__name__ == "add_vararg":
            return rf"({c})"
        return c
    d = _latex_derivative(context, dt.derivative)
    c = _latex_coeff(context, dt.coefficient)
    return f"{c} {d}"

In [13]:
x, y = var("x y")
w = function("w")(x, y)
z = function("z")(x, y)
l = [w,z,
     diff(w, x), diff(z, x),
     diff(w, y), diff(z, y),
     diff(w, x, x), diff(z, x, x),
     diff(w, x, y), diff(z, x, y),
     diff(w, y, y), diff(z, y, y),
     
    ]
ctx_grevlex=Context((w, z), (y, x), Mgrevlex)
#ctx_grlex=Context((w, z), (y, x), Mgrlex)
#ctx_lex=Context((w, z), (y, x), Mlex)
l = [gen_dt(ctx_grevlex, _) for _ in l]
shuffle(l)
l.sort()
#for _ in l:
#    display(Math(latexer_dt(ctx_grevlex, _)))
#    print(_.comparison_order)

In [14]:
pprint(l)

[DT(context=<delierium.MatrixOrder.Context object at 0x7f702fc8bbd0>,
    expression=z(x, y),
    coefficient=1,
    derivative=z(x, y),
    function=() |--> z(x, y),
    order=[0, 0],
    comparison_order=[0, 0, 0, 1]),
 DT(context=<delierium.MatrixOrder.Context object at 0x7f702fc8bbd0>,
    expression=w(x, y),
    coefficient=1,
    derivative=w(x, y),
    function=() |--> w(x, y),
    order=[0, 0],
    comparison_order=[0, 0, 1, 0]),
 DT(context=<delierium.MatrixOrder.Context object at 0x7f702fc8bbd0>,
    expression=diff(z(x, y), x),
    coefficient=1,
    derivative=diff(z(x, y), x),
    function=z,
    order=[0, 1],
    comparison_order=[0, 1, 0, 1]),
 DT(context=<delierium.MatrixOrder.Context object at 0x7f702fc8bbd0>,
    expression=diff(z(x, y), y),
    coefficient=1,
    derivative=diff(z(x, y), y),
    function=z,
    order=[1, 0],
    comparison_order=[1, 0, 0, 1]),
 DT(context=<delierium.MatrixOrder.Context object at 0x7f702fc8bbd0>,
    expression=diff(w(x, y), x),
    c

In [15]:
gen_dt(ctx, 2*A_x*B_x*xi_xy)

DT(context=<delierium.MatrixOrder.Context object at 0x7f703bbf0b90>, expression=2*diff(A(x, y), x)*diff(B(x, y), x)*diff(xi(x, y), x, y), coefficient=2*diff(A(x, y), x)*diff(B(x, y), x), derivative=diff(xi(x, y), x, y), function=xi, order=[1, 1], comparison_order=[1, 1, 1, 0])

In [16]:
class DP:
    def __init__(self, context, expression):
        self.dts = []
        self.context = context
        self.expression = expression
        if self.expression == 0:
            self.lterm = 0
            return
        set_trace()
        self.dts = list(map(lambda _: gen_dt(context, _), gen_summands(context, self.expression.expand().simplify())))
        self.dts.sort(reverse=True)
        counts = Counter(tuple(_.comparison_order) for _ in self.dts)
        while duplicates := [_ for _ in counts.items() if _[1] > 1]:
            indices = [i for i, dt in enumerate(self.dts) if tuple(dt.comparison_order) == duplicates[0][0]]
            dt = self.dts[indices[0]]
            for i in indices[1:]:
                dt.coefficient += self.dts[i].coefficient
            dt.expression = (dt.coefficient * dt.derivative).simplify()
            self.dts = [dt for i, dt in enumerate(self.dts) if i not in indices[1:]]
            counts = Counter(tuple(_.comparison_order) for _ in self.dts)
        lcoeff = self.dts[0].coefficient
        for _ in self.dts:
            try:
                _.coefficient = (_.coefficient / lcoeff).expand()
            except AttributeError:
                pass
        self.lder = self.dts[0].derivative
        self.comp = self.dts[0].comparison_order
        self.lfunc = self.dts[0].function
        self.order = self.dts[0].order
        assert (self.dts[0].coefficient == 1)
        self.lterm = self.dts[0]
        self.expression = sum(_.derivative * _.coefficient for _ in self.dts)
        print("B"*50)
        for _ in self.dts:
            print(f"{_=}")
        self.expression.expand().show()
        print("C"*50)
        
    def __lt__ (self, other):
        if not isinstance(other, DP):
            return False
        return self.lterm < other.lterm
    def __eq__(self, other):
        # XXX get rid of this check, check at source
        if not isinstance(other, DP) or not isinstance(self, DP):
            return False
        return self.lterm == other.lterm
    def __gt__(self, other):
        if not isinstance(other, DP):
            return False
        return self.lterm > other.lterm

    def __str__(self):
        return " + ".join((str(_) for _ in self.dts)).replace("+ -", "-")
    def __zero__(self):
        return not self.dts or self.expression == 0

In [17]:
def _Janet_Basis(dps):
    return sorted(dps)

In [ ]:
def latexer_dp(context , dp):
    return "+".join(map(lambda _ : latexer_dt(context, _), dp.dts)).replace("(-", "-(").replace("+-", "-")

#dp = DP(ctx_grevlex, (x+y**2)*diff(w,y,y,y,x) + diff(z, x,9))
#display(Math(latexer_dp(ctx_grevlex, dp)))

x, y=var("x y")
z=function("z")
w=function("w")
#h1=DP(ctx_grevlex,diff(z(x, y), y) -4*y/(x*(1/(x^2 + y) - 4*x/(y*(x/((x^2 + y)*y) - 2)) - 4*x^2/((x^4 + 2*x^2*y + y^2)*y*(x/((x^2 + y)*y) - 2)) + x/((x^2 + y)*y^2*(x/((x^2 + y)*y) - 2)) + 2/((x^2 + y)*y*(x/((x^2 + y)*y) - 2)))) * diff(z(x, y), x) -2/(x*(1/(x^2 + y) - 4*x/(y*(x/((x^2 + y)*y) - 2)) - 4*x^2/((x^4 + 2*x^2*y + y^2)*y*(x/((x^2 + y)*y) - 2)) + x/((x^2 + y)*y^2*(x/((x^2 + y)*y) - 2)) + 2/((x^2 + y)*y*(x/((x^2 + y)*y) - 2)))) * w(x, y))
#_h2=diff(w(x, y), y) + 1/2*x/((x^2 + y)*y) * diff(z(x, y), y) -1/y * w(x, y)
#_h2.show()
#h2=DP(ctx_grevlex, _h2)
_h3=diff(z(x, y), x, x) -1/2*x/(x^2 + y) + 1/2*x^2/((x^2 + y)*y) + 1/2/(x^2 + y) - 1/2*y/((x^2 + y)*x) * diff(w(x, y), x) -1/2*x^3/((x^4 + 2*x^2*y + y^2)*(x^2 + y)) - 1/2*x*y/((x^4 + 2*x^2*y + y^2)*(x^2 + y)) + 1/4*x/(x^2 + y)^2 + 1/8/(x^2 + y)^2 + 1/4*y/((x^2 + y)^2*x) * diff(z(x, y), y) -1/2*y/((x^2 + y)*x) * diff(z(x, y), x) -1/4/((x^2 + y)*x) * w(x, y)
_h3.show()
h3=DP(ctx_grevlex,_h3)

h4=DP(ctx_grevlex,diff(z(x, y), x, y) -1/2/(x^2 + y) * diff(z(x, y), y) + 2*y/x * diff(z(x, y), x) + 1/x * w(x, y))
h5=DP(ctx_grevlex,diff(z(x, y), y, y) -x^2/(x^4 + 2*x^2*y + y^2) - x*y/(x^2 + y) - x^2/((x^2 + y)*y) - y/(x^4 + 2*x^2*y + y^2) - y^2/((x^2 + y)*x) - 1/(x^2 + y) * diff(z(x, y), y) + 4*y^2 + 4*y^3/x^2 * diff(z(x, y), x) + 2*y + 2*y^2/x^2 * w(x, y))

-1/2*x^3/((x^4 + 2*x^2*y + y^2)*(x^2 + y)) - 1/2*x/(x^2 + y) + 1/2*x^2/((x^2 + y)*y) - 1/2*x*y/((x^4 + 2*x^2*y + y^2)*(x^2 + y)) - 1/2*y*diff(w(x, y), x)/((x^2 + y)*x) - 1/2*y*diff(z(x, y), x)/((x^2 + y)*x) + 1/2/(x^2 + y) + 1/4*x/(x^2 + y)^2 - 1/4*w(x, y)/((x^2 + y)*x) + 1/4*y*diff(z(x, y), y)/((x^2 + y)^2*x) + 1/8/(x^2 + y)^2 + diff(z(x, y), x, x)

> /tmp/ipykernel_15701/921131857.py(10)__init__()
      8             return
      9         set_trace()
---> 10         self.dts = list(map(lambda _: gen_dt(context, _), gen_summands(context, self.expression.expand().simplify())))
     11         self.dts.sort(reverse=True)
     12         counts = Counter(tuple(_.comparison_order) for _ in self.dts)



In [ ]:
def _reduce_inner(e1:DP, e2:DP, context:Context) -> DP:
    """
    Algorithm 2.4 from Schwarz, p.48
    
    >>> # Example 2.33(f1), p.48
    >>> x, y = var(\"x y\")
    >>> z = function(\"z\")(x,y)
    >>> ctx = Context([z], [x,y])
    >>> e1 = DP(ctx, diff(z,y) - (x**2)*diff(z, x)/(y**2) - (x-y)*z/(y**2))
    >>> f1 = DP(ctx, diff(z,x) + z/x)
    >>> print(_reduce_inner(e1, f1, ctx))
    diff(z(x, y), y) + 1/y * z(x, y)
    >>> # Example 2.33(f2), p.48
    >>> x, y = var("x y")
    >>> z = function("z")(x,y)
    >>> ctx = Context([z], [x,y])
    >>> e1 = DP(ctx, diff(z,y) - (x**2)*diff(z, x)/(y**2) - (x-y)*z/(y**2))
    >>> f2 = DP(ctx, diff(z,y) + z/y)
    >>> print(_reduce_inner(e1, f2, ctx))
    diff(z(x, y), x) + 1/x * z(x, y)
    """
    print("I"*99)
    print("e1:")    
    display(Math(latexer_dp(context, e1)))
    print("e2:")
    display(Math(latexer_dp(context, e2)))
    for t in (_ for _ in e1.dts if _.function == e2.lfunc):
        print(f"{str(t)=}")
        display(Math(latexer_dt(context,t)))
        # S1 from Algorithm 2.4
        dif = [a - b for a, b in zip(t.order, e2.order)]
        if all(map(lambda h: h == 0, dif)):
            # S2 from Algorithm 2.4
            print(f"subtraction: e1-e2, {str(t)=}, {str(t.coefficient)=}")
            try:
                t.coefficient.expand().simplify().show()
            except Exception as ex:
                pass
            return DP(context, e1.expression - e2.expression * t.coefficient)
        if all(map(lambda h: h >= 0, dif)):
            # S2 from Algorithm 2.4
            # toDo: as diff also accepts zeroth derivatives we
            # may unify these two branches
            #
            # ToDo: take care about the order of variables and              
            # order in context!
            variables_to_diff = []
            # XXX: this may be a problem: which index? that                 
            # from context or from order?
            #set_trace()            
            for i in range(len(context._independent)):
                if dif[i] != 0:
                    variables_to_diff.extend([context._independent[i]] * abs(dif[i]))
            print(f"differentiation: e1-e2, {str(t)=},{variables_to_diff=}, {str(t.coefficient)=}")
            return DP(context, e1.expression - t.coefficient * diff(e2.expression, *variables_to_diff))
    print("NOPE")
    return e1

x, y = var("x y")
z = function("z")(x,y)
ctx = Context([z], [x,y])
e1 = DP(ctx, diff(z,y) - (x**2)*diff(z, x)/(y**2) - (x-y)*z/(y**2))
f1 = DP(ctx, diff(z,x) + z/x)
f2 = DP(ctx, diff(z, y) + z/y)
#display(Math(latexer_dp(ctx, e1)))
#display(Math(latexer_dp(ctx, f1)))
#display(Math(latexer_dp(ctx, f2)))
r1 = _reduce_inner(e1, f1, ctx)
r2 = _reduce_inner(e1, f2, ctx)
display(Math(latexer_dp(ctx, r1)))
display(Math(latexer_dp(ctx, r2)))


#doctest.testmod()   

In [ ]:
def _reduce(e1:DP, e2:DP, context: Context) -> DP:
    _e1 = _reduce_inner(e1, e2, context)
    while not _e1 is e1:
        e1 = _e1
        _e1 = _reduce_inner(e1, e2, context)
    return _e1

In [ ]:
def reduceS(e: DP, S: list[DP], context: Context) -> DP:
    """Algorithm 2.5, p. 48
    >>> # example 2.34, p. 48
    >>> x, y = var('x y')
    >>> z = function('z')(x, y)
    >>> ctx = Context([z], (y, x))
    >>> e1 = DP(ctx, diff(z, y) - x**2 * diff(z, x)/y**2  - (x - y)*z/y**2)
    >>> f1 = DP(ctx, diff(z, x) + z/x)
    >>> f2 = DP(ctx, diff(z, y) + z/y)
    >>> print(reduceS(e1, [f1, f2], ctx))
    <BLANKLINE>
    >>> # example 2.35, p.48
    >>> vars = var ("x y")
    >>> z = function("z")(*vars)
    >>> w = function("w")(*vars)
    >>> # ctx: items are in descending order
    >>> ctx_grevlex_f = Context((w, z), (y, x), Mgrevlex)
    >>> ctx_grlex_f   = Context((w,z), vars, Mgrlex)
    >>> ctx_lex_f     = Context((w,z), vars, Mlex)
    >>> f1 = diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w/y
    >>> f2 = diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
    >>> f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
    >>> f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x*diff(z, y)/y - w/(2*y**2)
    >>> f5 = diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w/(y**2)
    >>> ctx = ctx_grevlex_f
    >>> r = reduceS(DP(ctx, f4), [DP(ctx, f1),DP(ctx, f2)], ctx)
    >>> den=x*(4*x**5*y + 8*x**3*y**2 - x**3 + 2*(x*y)**2 + 2*x**2*y +
    ... 4*x*y**3 - 2*x*y + 2*y**3-2*y**2)
    >>> r.expression - diff(z, y) + diff(z, x)*(4*(2*x**2 * y) - x + 2*y**2)*(x**2+y)*y**2/den - w*(2*x**2*y-x+2*y**2)*(x**2+y)*y/den
    0
    """
    reducing = True
    gen = [_ for _ in S]
    while reducing:
        for dp in gen:
            enew = _reduce(e, dp, context)
            print("*************************")
            for _ in enew.dts:
                print("........")
                print(str(_))
                try:
                    _.coefficient.numerator().expand().simplify().show()
                    _.coefficient.denominator().expand().simplify().show()
                except Exception as exp:
                     print(f"===>{_.coefficient=}")
            # XXX check whether we can replace "==" by 'is'
            if enew is e:
                reducing = False
            else:
                if enew == 0 or not str(enew):
                    return enew
                e = enew
                print(f"{str(enew)=}")
                display(Math(latexer_dp(context, enew)))
                gen = [_ for _ in S]
                reducing = True
    return enew
#doctest.testmod() 

In [ ]:
# example 2.34, p. 48
x, y = var('x y')
z = function('z')(x, y)
ctx = Context([z], (x,y))
#e1 = DP(ctx, diff(z, y) - x**2 * diff(z, x)/y**2  - (x - y)*z/y**2)
#f1 = DP(ctx, diff(z, x) + z/x)
#f2 = DP(ctx, diff(z, y) + z/y)
#print(e1.dts)
#print(f1.dts)
#print(f2.dts)
#r = reduceS(e1, [f1, f2], ctx)
#print(f"{str(r)=}")

# example 2.35, p.48
vars = var ("x y")
z = function("z")(*vars)
w = function("w")(*vars)
# ctx: items are in descending order
ctx_grevlex_f = Context((w, z), (y, x), Mgrevlex)
ctx_grlex_f   = Context((w,z), vars, Mgrlex)
ctx_lex_f     = Context((w,z), vars, Mlex)

f1 = diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w/y
f2 = diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w/(2*y**2)
f5 = diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w/(y**2)
ctx = ctx_grevlex_f
r = reduceS(DP(ctx, f4), [DP(ctx, f1),DP(ctx, f2)], ctx)
print(r, r.__class__)

In [ ]:
for _ in r.dts: _.coefficient.denominator().expand().show()

In [ ]:
for _ in r.dts: _.coefficient.numerator().expand().show()

In [ ]:
r.expression.simplify().expand().show()

In [ ]:
def autoreduce(context, dps):
    i =  0
    _p, r = dps[:i + 1], dps[i + 1:]
    count = 0
    while r:
        count += 1
        print(f"========== {count=} =========")
        newdps = []
        have_reduced = False
        for _r in r:
            rnew = reduceS(_r, _p, context)
            have_reduced = have_reduced or rnew != _r
            if rnew:
                newdps.append(rnew)
        dps = list(sorted(_p + [_ for _ in newdps if _.dts and _ not in _p], reverse = False))
        # XXX third iteration is sorted wrong, first and second must
        # be interchanged
        print("A"*88)        
        for _x in dps:
            display(Math(latexer_dp(context, _x)))
        set_trace()
        if not have_reduced:
            i += 1
        else:
            i = 0
        _p, r = dps[:i + 1], dps[i + 1:]
    return dps

class Janet_Basis:
    def __init__(self, context: Context, expressions: list[_Sage_Expression]):
        self.dps = sorted([DP(context, _) for _ in expressions],reverse = False)
        #for _ in self.dps:
        #    display(Math(latexer_dp(context, _)))
#        set_trace()
        self.dps = autoreduce(context, self.dps)

In [ ]:
vars = var ("x y")
z = function("z")(*vars)
w = function("w")(*vars)
# ctx: items are in descending order
ctx_grevlex_f = Context((w, z), (y, x), Mgrevlex)
ctx_grlex_f   = Context((w,z), vars, Mgrlex)
ctx_lex_f     = Context((w,z), vars, Mlex)

f1 = diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w/y
f2 = diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x
f3 = diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2
f4 = diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w/(2*y**2)
f5 = diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w/(y**2)
system_2_24 = [f1,f2,f3,f4,f5]
system_2_24.sort()
for _ in system_2_24:
    display(Math(latexer_dp(ctx_grevlex_f, DP(ctx_grevlex_f, _))))

In [ ]:
jb = Janet_Basis(ctx_grevlex_f, system_2_24)

In [ ]:
for _ in jb.dps:
    print(\"===\")
    for __ in _.dts:
        try:
            __.coefficient.simplify_full().show()
       except:
            pass

In [ ]:
vars = var ("x y")
z = function("z")(*vars)
w = function("w")(*vars)
# ctx: items are in descending order
ctx_grevlex_f = Context((w, z), (y,x), Mgrevlex)
ctx_grlex_f   = Context((w,z), vars, Mgrlex)
ctx_lex_f     = Context((w,z), vars, Mlex)

f1 = diff(z, y)
f2 = diff(z,y,y)
f3 = diff(z, x,x)
f4 = diff(z, x,y)
f5 = diff(w,y)
for _ in list(sorted([DP(ctx_grevlex_f, _) for _ in [f1,f2,f3,f4,f5]], reverse = False)):
    display(Math(latexer_dp(ctx_grevlex_f, _)))

In [ ]:
vars = var ("x y")
z = function("z")(*vars)
w = function("w")(*vars)
# ctx: items are in descending order
ctx_grevlex_f = Context((w,z), (y,x), Mgrevlex)
ctx_grlex_f   = Context((w,z), vars, Mgrlex)
ctx_lex_f     = Context((w,z), vars, Mlex)

g1 = diff(z, y,y) + diff(z, y)/(2*y)
g2 = diff(w, x, x) + 4*y**2*diff(w, y)-8*y**2*diff(z, x)-8*y*w
g3 = diff(w, x,y) - x*diff(z, x,2)/2 - diff(w, x)/(2*y) - 6*y**2*diff(z,y)
g4 = diff(w, y, y) -2* diff(z, x,y) - diff(w, y)/(2*y) + w/(2*y**2)
    
system_2_25 = [g2, g3, g4, g1]
jb = Janet_Basis(ctx_grevlex_f, system_2_25)

In [ ]:
# now ist the time to check the sorting....\n",
    #"x, y, z = var(\"x y z\")\n",
    #"u = function (\"u\")(x,y,z)\n",
    #"ctxMlex = Context((u,),(x,y,z), Mlex)\n",
    #"ctxMgrlex = Context((u,),(x,y,z), Mgrlex)\n",
    #"ctxMgrevlex = Context((u,),(x,y,z), Mgrevlex)\n",
    #"# this is the standard example stolen from wikipedia\n",
    #"u0 = gen_dt(ctxMlex, diff(u,x,3))\n",
    #"u1 = gen_dt(ctxMlex, diff(u,z,2))\n",
    #"u2 = gen_dt(ctxMlex, diff(u,x,y,y,z))\n",
    #"u3 = gen_dt(ctxMlex, diff(u, x,x,z,z))\n",
    #"l1 = [u0, u1,u2,u3]\n",
    #"shuffle(l1)\n",
    #"s = list(sorted(l1, reverse = True))\n",
    #"for _ in s: print(_.order)\n",
    #diff(u(x, y, z), z, z)\n",
    #diff(u(x, y, z), x, y, y, z)\n",
    #diff(u(x, y, z), x, x, z, z)\n",
    #diff(u(x, y, z), x, x, x)\n",
    #        >>> s = sorted(l1, key=cmp_to_key(lambda item1, item2: item1.sorter(item2)))\n",
    #        >>> for _ in s: print(_)\n",
    #        diff(u(x, y, z), z, z)\n",
    #        diff(u(x, y, z), x, x, x)\n",
    #        diff(u(x, y, z), x, y, y, z)\n",
    #        diff(u(x, y, z), x, x, z, z)\n",
    #        >>> s = sorted(l1, key=cmp_to_key(lambda item1, item2: item1.sorter(item2)))\n",
    #        >>> for _ in s: print(_)\n",
    #        diff(u(x, y, z), z, z)\n",
    #        diff(u(x, y, z), x, x, x)\n",
    #        diff(u(x, y, z), x, x, z, z)\n",
    #        diff(u(x, y, z), x, y, y, z)\n",
    #        '''"
   #]
# "x, y = var('x y')\n",
#    "w = function('w')(x, y)\n",
#    "z = function('z')(x, y)\n",
#    "ctx = Context((w, z), (y, x), Mgrlex)\n",
#    "print(ctx._weight)\n",
#    "l = [gen_dt(ctx, z), gen_dt(ctx, diff(z, x)), gen_dt(ctx, diff(z ,y)), gen_dt(ctx, diff(z ,x,x)), gen_dt(ctx, diff(z ,x, y)), gen_dt(ctx, diff(z ,y, y))] + \\\n",
#    "    [gen_dt(ctx, w), gen_dt(ctx, diff(w, x)), gen_dt(ctx, diff(w ,y)), gen_dt(ctx, diff(w ,x,x)), gen_dt(ctx, diff(w ,x, y)), gen_dt(ctx, diff(w ,y, y))]\n"